In [21]:
from skimage import io
from skimage.color import rgb2gray
from skimage.transform import rescale
from matplotlib import pyplot as plt
from bresenham import bresenham
import warnings
import pydicom
import numpy as np
from math import floor, ceil, sqrt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import IPython


# Questions
# 1. Does bresenham algorithm has to be implemented by ourselves? YES
# 2. What kind of patient's data can be editted by user? imie, nazwisko, PESEL

def reverse_radon(sinogram, pixels_lines_count, emitters, detectors):
    print("Processing sinogram to image...")
    image = np.array([])
    detectors_count = sinogram.shape[0]
    steps = sinogram.shape[1]
    height = pixels_lines_count.shape[0]
    width = pixels_lines_count.shape[1]

    image = np.zeros(( height, width ))
    
    for i in range( steps ):
        for j in range(detectors_count):
            coordinates = list(bresenham(emitters[i][0], emitters[i][1], detectors[i][j][0], detectors[i][j][1]))
            for c in coordinates:
                if c[0] < height and c[1] < width and c[0] >= 0 and c[1] >= 0: 
                    image[c[0], c[1]] += sinogram[j, i] / pixels_lines_count[c[0], c[1]]
    
    plt.imshow(image, 'gray')
    plt.show()

def degrees_to_radians(alpha, arc):
    return np.deg2rad(alpha), np.deg2rad(arc)
    
def count_image_parameters(image):
    height = image.shape[0]
    width = image.shape[1]
    center = ( floor( height/2 ), floor( width/2 ))
    pixels_lines_count = np.zeros((height, width))
    radius = ceil( sqrt(height**2 + width**2) / 2 )
    return height, width, center, radius, pixels_lines_count

def load_image(path):
    image = rgb2gray(io.imread(path, multichanel = False))
    image = rescale(image, 1.0 / 5.0, anti_aliasing=False, mode='constant')
    print("Image to process:")
    plt.imshow(image, 'gray')
    plt.show()
    return(image)

def prepare_output():
    warnings.filterwarnings('ignore')
    IPython.display.clear_output()
    
@interact(alpha=(1,30,1), arc = (5, 180, 5), detectors_count=(10, 300, 10))
def radon(alpha, arc, detectors_count):
    prepare_output()
    image = load_image('brain.jpg')
    height, width, center,  radius, pixels_lines_count = count_image_parameters(image)
    alpha, arc = degrees_to_radians(alpha, arc)
    print(alpha)
    print(arc)
    steps = int( np.pi*2 / alpha)
    sinogram = np.zeros((detectors_count, steps))
    emitter_coordinates = []
    detector_coordinates = []
    print("Sinogram processing")
    for i in range( steps ):
        angle = alpha * i
        x_emitter = center[0] - int(radius * np.cos(angle))
        y_emitter =  center[1] - int(radius * np.sin(angle))
        # if x_emitter < width and y_emitter < height:
        #     image[x_emitter, y_emitter] = 1
        emitter_coordinates.append((x_emitter, y_emitter))
        detector_coordinates.append([])

        for j in range(detectors_count):
            x = center[0] - int(radius * np.cos(angle + np.pi - arc/2 + j * arc/(detectors_count-1)))
            y = center[1] - int(radius * np.sin(angle + np.pi - arc/2 + j * arc/(detectors_count-1)))
            coordinates = list(bresenham(x_emitter, y_emitter, x, y))
            detector_coordinates[i].append((x,y))
            
            for c in coordinates:
                if c[0] < height and c[1] < width and c[0] >= 0 and c[1] >= 0: 
                    image[c[0], c[1]]= 1
                    sinogram[j, i] += image[c[0], c[1]]
                    pixels_lines_count[c[0], c[1]] += 1
    
    plt.imshow(sinogram, 'gray')
    plt.show()
    
    reverse_radon(sinogram, pixels_lines_count, emitter_coordinates, detector_coordinates)




# # DICOM files management
# ds = pydicom.dcmread("0015.DCM")
# print(ds)
# print(ds.PatientName)
# print(ds.PatientID)
# print(ds.StudyDate)
# print(ds.ImageComments) 

# plt.imshow(ds.pixel_array, cmap = 'gray') 
# plt.show()